# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096888


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:19,  2.83s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:55,  2.06s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:51,  1.99s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:38,  1.53s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:13<00:01,  3.38it/s]

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.61it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  4.12it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.94it/s]

not-logged-in-3b0b929f4475fd276561    0.000481
Jnursssmith5263                       0.000977
LEO400                                0.000170
Kartoon1x                             0.000245
jnarayanbvg                           0.046851
Jonas_Cross                           0.000612
not-logged-in-c6917d9b19f41e6c2995    0.010165
gracie_solveig                        0.000254
mememan                               0.226153
not-logged-in-c6917d9b19f41e6c2995    0.000952
not-logged-in-4ddf669750977b537491    0.001006
kc_                                   0.002303
BraxtonKinney                         0.000228
Lavadude                              0.061148
not-logged-in-a727525c505a13057531    0.066224
fanboy_deluxe                         0.002671
Denny2507                             0.000460
niiler                                0.000303
Jassels2                              0.000497
not-logged-in-3429675089c67c4fccb8    0.022070
olmy.007                              0.000329
mhaloftis    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())